In [1]:
using Pkg

projectdir_static = dirname(Base.active_project())

projectdirx(args...) = joinpath(projectdir_static, args...)

# Generate functions to access the path of default subdirectories.
for dir_type ∈ ("data", "src", "plots", "scripts", "papers")
    function_name = Symbol(dir_type * "dirx")
    @eval begin
        $function_name(args...) = projectdirx($dir_type, args...)
    end
end

using DrWatson

@quickactivate "TissueRigidity"
 
projectname()

"TissueRigidity"

In [2]:
using DifferentialEquations
using StatsBase
using CairoMakie
using XLSX
using DataFrames
# using Optimization, OptimizationPolyalgorithms, SciMLSensitivity,OptimizationOptimJL,OptimizationBBO,OptimizationNOMAD

In [3]:
IPNewton

UndefVarError: UndefVarError: IPNewton not defined

In [ ]:
include(srcdirx("Plotting&Utility.jl"))
include(srcdirx("FittingFunctions.jl"))
# include(srcdirx("old/FittingFunctions_old_cmax.jl"))

include(srcdirx("NodalLefty_E.jl"))

exp_name = "NodalLefty_RelayDiffusion_Optimal_v2"

include(srcdirx("ExperimentSetups/" * exp_name * ".jl"))

include(scriptsdirx("LoadData.jl"));

In [ ]:
α_range = 0.7:0.01:0.95

fig = Figure(size = (500,300))
ax1 = Axis(fig[1,1],title = "Porosity",ylabel = L"\phi(\alpha)",xlabel = L"\alpha")

lines!(ax1, α_range,[ϕ(α) for α in α_range])

fig

In [ ]:
pv_orig = [DN0,DL0,kN0,kL0,kE,kNL,σN0,σL0,Na,NL,NE,LN,s0]
# pv_orig = load(datadirx() * "/sims/NodalLefty_DiffusionDominated_OptParmas.jld2")["OptimalParam"][3]

p_orig,p_cp_orig,p_lm_orig = get_params(pv_orig)

tspan = (0,Inf)

u0 = ones(Nc,4)

u0[:,1] .= 1e-10
u0[:,2] .= 1e-10
u0[:,3] .= 0.
u0[:,4] .= α0

prob = ODEProblem(nodal_lefty_spatial_diff!,u0,tspan,p_orig)

lb = copy(pv_orig)
ub = copy(pv_orig)

lb[1:2] = 0.9 .* lb[1:2]
ub[1:2] = 1.1 .* ub[1:2]

lb[3:end] = (1 - γ) .* lb[3:end]
ub[3:end] = (1 + γ) .* ub[3:end];

In [ ]:
orig_metrics = get_summary_metrics(pv_orig,prob,data,alpha_data,0.2)

In [ ]:
fig = Figure(size = (500,300))

ax = Axis(fig[1,1], xlabel = L"\text{time (mins)}", ylabel= L"\lambda_{1/2} \quad (\mu m)",ygridvisible = false,xgridvisible = false,ylabelcolor = :blue,yticklabelcolor = :blue)
ax_N = Axis(fig[1,1], xlabel = L"\text{Rescaled time, t}", ylabel= L"N(x=0,t)", yaxisposition = :right,ylabelcolor = :green,yticklabelcolor = :green,ygridvisible = false,xgridvisible = false,xticksvisible = false)

hidexdecorations!(ax_N)

sol = solve(prob, p = p_orig, FBDF(),abstol = de_abstol,reltol = de_reltol, maxiters = 1e6,callback = TerminateSteadyState(1e-6,1e-4),isoutofdomain = (u,p,t) -> any(x->x<0, u));

N_samp = 5000

λ_trange = LinRange(0.,sol.t[end],N_samp)

N0t = [sol(t)[1,1] for t in λ_trange]

λhalf,λhalf_max_t = get_lambda_half(sol,λ_trange)

c_max = maximum(sol(λhalf_max_t)[:,1])

lines!(ax,λ_trange ./ 60 ,λhalf,color = :blue)

lines!(ax_N,λ_trange ./ 60 ,N0t, color = :green)

vlines!(ax,λhalf_max_t / 60, color = :red, linestyle = :dash)

save(plotsdir(exp_name * "/OrigFits/LambdaHalfOverT.pdf"),fig)

fig

In [ ]:
fig = Figure(size = (1200,1200))

# fig = plot_summary!(fig,[DN0,DL0,kN0,kL0,2*kE,0.5*kNL,σN0,0.9*σL0,1.2*Na,NL,NE,LN,s0],prob)

# fig = plot_summary!(fig,[DN0,DL0,0.5*kN0,kL0,2*kE,0.5*kNL,σN0,0.7*σL0,1.5*Na,NL,0.3*NE,0.5*LN,s0],prob)

fig = plot_summary!(fig,[DN0,DL0,kN0,kL0,kE,kNL,σN0,σL0,Na,NL,NE,LN,s0],prob)

save(plotsdir(exp_name * "/OrigFits/Summary_orig.pdf"),fig)

fig


In [ ]:
# for i in 1:length(pv_orig)
#     print(i)
#     for pt in [0.8,1.2]
#         fig = Figure(size = (1200,1200))
#         p_try = copy(pv_orig)
#         p_try[i] = p_try[i]*pt
#         plot_summary!(fig,p_try,prob)
#         save(plotsdir(exp_name * "/Perturb/08/" * p_names_string[p_names[i]] * "_" * string(pt) * ".pdf"),fig)
#     end
# end

In [ ]:
# for i in 1:length(pv_orig)
#     print(i)
#     for pt in [0.5,1.5]
#         fig = Figure(size = (1200,1200))
#         p_try = copy(pv_orig)
#         p_try[i] = p_try[i]*pt
#         plot_summary!(fig,p_try,prob)
#         save(plotsdir(exp_name * "/Perturb/05/" * p_names_string[p_names[i]] * "_" * string(pt) * ".pdf"),fig)
#     end
# end

In [ ]:
sim_data = load(datadirx() * "/sims/" * exp_name * "_CPSet_Sweep_RestrID_nosig.jld2")["Results"];
sim_data_p = load(datadirx() * "/sims/" * exp_name * "_CPSet_Sweep_RestrID_nosig.jld2")["Parameters"];
res_id = findall(x->length(x) == 3 ,sim_data)

sim_data = sim_data[res_id]
sim_data_p =sim_data_p[res_id];

In [ ]:
sim_data_02 = map(x->x[3],sim_data);

In [ ]:
# sim_data_02_c1_id = findall(x-> (x[:wt_xMax] < x[:cp_xMax]) & (x[:wt_lprod_t0 ] < x[:cp_lprod_t0]) & (x[:cp_d0] < 0.5) & (x[:inc_met] < 10.) ,sim_data_02); P1

sim_data_02_c1_id = findall(x-> (x[:wt_xMax] < x[:cp_xMax]) & (1.2*x[:wt_lprod_t0 ] < x[:cp_lprod_t0]) & (x[:wt_d0] < 0.2)  & (x[:cp_d0] < 0.7) & (x[:inc_met] == 0) ,sim_data_02); #P2 - 22

# sim_data_02_c1_id = findall(x-> (x[:wt_xMax] < x[:cp_xMax]) & (x[:wt_lprod_t0 ] < x[:cp_lprod_t0]) & (x[:inc_met] < 0.1) ,sim_data_02);

In [ ]:
id = argmin(map(x->x[:alpha_mse],sim_data_02[sim_data_02_c1_id]))

id = 1

# id = argmin(map(x->mean(x[:xmax_mse]),sim_data_02[sim_data_02_c1_id]))

In [ ]:
sim_data_02[sim_data_02_c1_id][id]

In [ ]:
p_vector = sim_data_p[sim_data_02_c1_id][id]

# p,p_cp,p_lm = get_params(p_vector)

# sol = solve(prob, p = p, FBDF(),abstol = de_abstol,reltol = de_reltol, maxiters = 1e6,callback = TerminateSteadyState(1e-6,1e-4),isoutofdomain = (u,p,t) -> any(x->x<0, u));
# sol_cp = solve(prob, p = p_cp, FBDF(),abstol = de_abstol,reltol = de_reltol,maxiters = 1e6,callback = TerminateSteadyState(1e-6,1e-4),isoutofdomain = (u,p,t) -> any(x->x<0, u));
# sol_lm = solve(prob, p = p_lm, FBDF(),abstol = de_abstol,reltol = de_reltol,maxiters = 1e6,callback = TerminateSteadyState(1e-6,1e-4),isoutofdomain = (u,p,t) -> any(x->x<0, u));

# λ_trange = LinRange(0.,sol.t[end],N_samp)

# λhalf,λhalf_max_t = get_lambda_half(sol,λ_trange)

# c_max_wt = maximum(sol(λhalf_max_t)[:,1])

In [ ]:
# c_level = 0.2*c_max_wt

# t_grid = LinRange(0,sol.t[end],t_grid_N)

# level_x_wt = get_level_x(sol,c_level,t_grid);
# level_x_cp = get_level_x(sol_cp,c_level,t_grid)

# wt_t0 = t_grid[argmax(level_x_wt)];
# cp_t0 = t_grid[argmax(level_x_cp)];

# t_plot = LinRange(0,exp_times_times_norm[end],t_plot_N)

# level_x_wt_rescaled = get_level_x(sol,c_level,t_plot .* wt_t0)  
# level_x_cp_rescaled = get_level_x(sol_cp,c_level,t_plot .* wt_t0)
# level_x_lm_rescaled  = get_level_x(sol_lm,c_level,t_plot .* wt_t0)

# wt_xMax = maximum(level_x_wt_rescaled)
# cp_xMax = maximum(level_x_cp_rescaled)
# lm_xMax = maximum(level_x_lm_rescaled)

# wt_d0 = level_x_wt_rescaled[end] ./ wt_xMax
# cp_d0 = level_x_cp_rescaled[end] ./ cp_xMax
# lm_d0 = level_x_lm_rescaled[end] ./ lm_xMax

In [ ]:
fig = Figure(size = (1200,1200))
ax = Axis(fig[1,1])
lines!(ax,[sol.u[1]*exp(sol.u[2]*x) for x in tissue])
scatter!(ax,dynN)

fig

In [ ]:
orig_metrics = get_summary_metrics_cpset(p_vector,prob,data,alpha_data,[0.2])

In [ ]:
sim_tuple, sim_cp_tuple,_ = get_params(p_vector)

In [ ]:
fig = Figure(size = (1200,1200))

fig = plot_summary!(fig,[sim_tuple[:DN0],sim_tuple[:DL0],sim_tuple[:kN0],sim_tuple[:kL0],sim_tuple[:kE],sim_tuple[:kNL],sim_tuple[:σN0],sim_tuple[:σL0],sim_tuple[:Na],sim_tuple[:NL],sim_tuple[:NE],sim_tuple[:LN],sim_tuple[:s0]],prob)

fig

In [ ]:
save(plotsdir(exp_name * "/OptFits/Sweep_opt_2_alpha.pdf"),fig)

In [ ]:
sol = solve(prob, p = sim_tuple, FBDF(),abstol = de_abstol,reltol = de_reltol, maxiters = 1e6,callback = TerminateSteadyState(1e-6,1e-4),isoutofdomain = (u,p,t) -> any(x->x<0, u));
sol_cp = solve(prob, p = sim_cp_tuple, FBDF(),abstol = de_abstol,reltol = de_reltol,maxiters = 1e6,callback = TerminateSteadyState(1e-6,1e-4),isoutofdomain = (u,p,t) -> any(x->x<0, u));

In [ ]:
orig_metrics = get_summary_metrics(p_vector,prob,data,alpha_data,0.2)

t_plot_int = LinRange(0,3*orig_metrics[:wt_t0],1000)

nodal_prod_cp,nodal_prod = get_integrated_nodal_prod_values(sol,sol_cp,t_plot_int)


In [ ]:
fig = Figure(size = (500,500))

ax = Axis(fig[1,1], xlabel = L"\text{Rescaled time, } t^{'}", ylabel= L"\int_0^L ν(N(t,x)) dx" )

lines!(ax,LinRange(0,3,1000),nodal_prod_cp,linestyle = :dash,color = :grey, label = L"\text{Wnt11}")
lines!(ax,LinRange(0,3,1000),nodal_prod,color = :grey, label = L"\text{WT}")

axislegend(ax,position = :rt)

save(plotsdir(exp_name * "/OptFits/IntegratedNodal.pdf"),fig)

fig

In [ ]:
fig = Figure(size = (500,500))


ax = Axis(fig[1,2], xlabel = L"\text{t, mins}", ylabel= L"\int_0^L ν_L(N(t,x)) dx" )

lines!(ax,ncp,linestyle = :dash,color = :grey, label = L"\text{Wnt11}")
lines!(axνN_int,color = :grey, label = L"\text{WT}")

axislegend(ax,position = :rt)

In [ ]:
save(plotsdir(exp_name * "/OptFits/Summary_Sweep_P2.pdf"),fig)

In [ ]:
pv_orig = [DN0,DL0,kN0,kL0,2*kE,0.5*kNL,σN0,0.9*σL0,1.2*Na,NL,NE,LN,s0]

In [ ]:
for i in 1:length(p_base)
    print(i)
    for pt in [0.8,1.2]
        fig = Figure(size = (1200,1200))
        p_try = copy(p_base)
        p_try[i] = p_try[i]*pt
        plot_summary!(fig,p_try,prob)
        save(plotsdir(exp_name * "/PerturbOpt/" * p_names_string[p_names[i]] * "_" * string(pt) * ".pdf"),fig)
    end

end

In [ ]:
map(x->x[:cp_lprod_t0] - x[:wt_lprod_t0],result_dict[:Na])

In [ ]:
p_names_string[p_names[1]]